In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import re
import pprint

import folium
from folium.plugins import MarkerCluster
import googlemaps
import geopandas as gpd

import requests
import json
import time

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
bus_time_table = pd.read_csv('./datasets/bus_time_table.csv', encoding='cp949', sep=',', parse_dates=['기준_날짜'])
bus_time_table.isna().sum()

기준_날짜            0
행정동_ID           0
버스_승객_수        424
버스_승객_수_00시    424
버스_승객_수_01시    424
버스_승객_수_02시    424
버스_승객_수_03시    424
버스_승객_수_04시    424
버스_승객_수_05시    424
버스_승객_수_06시    424
버스_승객_수_07시    424
버스_승객_수_08시    424
버스_승객_수_09시    424
버스_승객_수_10시    424
버스_승객_수_11시    424
버스_승객_수_12시    424
버스_승객_수_13시    424
버스_승객_수_14시    424
버스_승객_수_15시    424
버스_승객_수_16시    424
버스_승객_수_17시    424
버스_승객_수_18시    424
버스_승객_수_19시    424
버스_승객_수_20시    424
버스_승객_수_21시    424
버스_승객_수_22시    424
버스_승객_수_23시    424
dtype: int64

* 결측치

In [3]:

del_idx = bus_time_table[bus_time_table.isna().values == True].index
bus_time_table = bus_time_table.drop(del_idx, axis=0)
bus_time_table.isna().sum()

기준_날짜          0
행정동_ID         0
버스_승객_수        0
버스_승객_수_00시    0
버스_승객_수_01시    0
버스_승객_수_02시    0
버스_승객_수_03시    0
버스_승객_수_04시    0
버스_승객_수_05시    0
버스_승객_수_06시    0
버스_승객_수_07시    0
버스_승객_수_08시    0
버스_승객_수_09시    0
버스_승객_수_10시    0
버스_승객_수_11시    0
버스_승객_수_12시    0
버스_승객_수_13시    0
버스_승객_수_14시    0
버스_승객_수_15시    0
버스_승객_수_16시    0
버스_승객_수_17시    0
버스_승객_수_18시    0
버스_승객_수_19시    0
버스_승객_수_20시    0
버스_승객_수_21시    0
버스_승객_수_22시    0
버스_승객_수_23시    0
dtype: int64

* 컬럼명 다시

In [4]:

bus_time_table[bus_time_table[['버스_승객_수_00시', '버스_승객_수_01시',
       '버스_승객_수_02시', '버스_승객_수_03시', '버스_승객_수_04시', '버스_승객_수_05시',
       '버스_승객_수_06시', '버스_승객_수_07시', '버스_승객_수_08시', '버스_승객_수_09시',
       '버스_승객_수_10시', '버스_승객_수_11시', '버스_승객_수_12시', '버스_승객_수_13시',
       '버스_승객_수_14시', '버스_승객_수_15시', '버스_승객_수_16시', '버스_승객_수_17시',
       '버스_승객_수_18시', '버스_승객_수_19시', '버스_승객_수_20시', '버스_승객_수_21시',
       '버스_승객_수_22시', '버스_승객_수_23시']].sum(axis=1) != bus_time_table['버스_승객_수']]

,기준_날짜,행정동_ID,버스_승객_수,버스_승객_수_00시,버스_승객_수_01시,버스_승객_수_02시,버스_승객_수_03시,버스_승객_수_04시,버스_승객_수_05시,버스_승객_수_06시,...,버스_승객_수_14시,버스_승객_수_15시,버스_승객_수_16시,버스_승객_수_17시,버스_승객_수_18시,버스_승객_수_19시,버스_승객_수_20시,버스_승객_수_21시,버스_승객_수_22시,버스_승객_수_23시


In [5]:

bus = pd.DataFrame()
bus['busdate'] = bus_time_table['기준_날짜']
bus['yy'] = bus_time_table['기준_날짜'].dt.year
bus['mm'] = bus_time_table['기준_날짜'].dt.month
bus['dd'] = bus_time_table['기준_날짜'].dt.day
bus['dong_id'] = bus_time_table['행정동_ID']
bus['guest_cnt'] = bus_time_table['버스_승객_수']
bus[['hh00', 'hh01',
       'hh02', 'hh03', 'hh04', 'hh05',
       'hh06', 'hh07', 'hh08', 'hh09',
       'hh10', 'hh11', 'hh12', 'hh13',
       'hh14', 'hh15', 'hh16', 'hh17',
       'hh18', 'hh19', 'hh20', 'hh21',
       'hh22', 'hh23']] = bus_time_table[['버스_승객_수_00시', '버스_승객_수_01시',
       '버스_승객_수_02시', '버스_승객_수_03시', '버스_승객_수_04시', '버스_승객_수_05시',
       '버스_승객_수_06시', '버스_승객_수_07시', '버스_승객_수_08시', '버스_승객_수_09시',
       '버스_승객_수_10시', '버스_승객_수_11시', '버스_승객_수_12시', '버스_승객_수_13시',
       '버스_승객_수_14시', '버스_승객_수_15시', '버스_승객_수_16시', '버스_승객_수_17시',
       '버스_승객_수_18시', '버스_승객_수_19시', '버스_승객_수_20시', '버스_승객_수_21시',
       '버스_승객_수_22시', '버스_승객_수_23시']].astype('int')
bus.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 173416 entries, 0 to 173839
Data columns (total 30 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   busdate    173416 non-null  datetime64[ns]
 1   yy         173416 non-null  int64         
 2   mm         173416 non-null  int64         
 3   dd         173416 non-null  int64         
 4   dong_id    173416 non-null  int64         
 5   guest_cnt  173416 non-null  float64       
 6   hh00       173416 non-null  int32         
 7   hh01       173416 non-null  int32         
 8   hh02       173416 non-null  int32         
 9   hh03       173416 non-null  int32         
 10  hh04       173416 non-null  int32         
 11  hh05       173416 non-null  int32         
 12  hh06       173416 non-null  int32         
 13  hh07       173416 non-null  int32         
 14  hh08       173416 non-null  int32         
 15  hh09       173416 non-null  int32         
 16  hh10       173416 no

* 인구자료

In [8]:
population = pd.read_csv('./datasets/population_2022_03.csv')

In [10]:
population.head(5)

,동별(1),동별(2),동별(3),2022. 03,2022. 03.1,2022. 03.2,2022. 03.3,2022. 03.4,2022. 03.5,2022. 03.6,2022. 03.7,2022. 03.8,2022. 03.9
0,동별(1),동별(2),동별(3),세대 (세대),합계 (명),합계 (명),합계 (명),한국인 (명),한국인 (명),한국인 (명),외국인 (명),외국인 (명),외국인 (명)
1,동별(1),동별(2),동별(3),소계,소계,남자,여자,소계,남자,여자,소계,남자,여자
2,합계,종로구,소계,73766,153684,74018,79666,144275,69911,74364,9409,4107,5302
3,합계,종로구,사직동,4679,9630,4335,5295,9370,4189,5181,260,146,114
4,합계,종로구,삼청동,1227,2741,1312,1429,2441,1168,1273,300,144,156
